In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True


# show all columns
pd.options.display.max_columns = None

In [2]:
allbuildings = pd.read_csv(".\\Inputs\\run207year2015allbuildingstest.csv")
newbuildings = pd.read_csv(".\\Inputs\\run207year2015bnewtest.csv")
parcels = pd.read_csv(".\\Inputs\\run207year2015parcelstest.csv")
zoning = pd.read_csv(".\\Inputs\\zoning_2015.csv")

In [3]:
base_year = 2015
year = 2016

### Inspect output tables

In [4]:
allbuildings.head()

,building_id,unit_price_non_residential,non_residential_price,note,residential_price,year_built,building_type_id,res_price_per_sqft,non_residential_sqft,stories,residential_units,parcel_id,building_sqft,job_spaces,node_id,zone_id,general_type,sqft_per_unit,lot_size_per_unit,sqft_per_job,vacant_residential_units,vacant_job_spaces,county_id,distlrg_id,distmed_id,distsml_id,distmed_1,distmed_2,distmed_3,distmed_4,distmed_5,distmed_6,distmed_7,distmed_8,distmed_9,distmed_10,distmed_11,distmed_12,distmed_13,distmed_14,distmed_15,distmed_16,distmed_17,distmed_18,distmed_19,distmed_20,distmed_21,distmed_22,distmed_23,distmed_24,distmed_25,distmed_26,distmed_27,distmed_28,distmed_29,distmed_30,distmed_31,distmed_32,distmed_33,distmed_34,distmed_35,distmed_36,distmed_37,distmed_38,distmed_39,distmed_40,distmed_41,zone_1212,building_age,real_far,land_value,utmxi,utmyi,avg_building_age,parcel_acres,parcel_volume,elevation,volume_two_way,volume_two_way_nofwy,max_far,residential_sqft,is_sf,is_mf,is_industrial,is_retail,is_office,is_govt,is_mixeduse,is_other,airport,airport_distance,fwy_exit_dist,fwy_exit_dist_tdm_output,raildepot_dist,university_dist,trail_dist,stream_dist,rail_stn_dist,bus_rte_dist,bus_stop_dist,residential_sales_price_sqft,total_job_spaces,rail_depot,improvement_value,unit_price_residential,distlrg_med_inc,distmed_med_inc,distsml_med_inc
0,65788,0.0,0,base,0,1963,1,84.432841,0.0,1.0,1.0,487357,936.000000,0.0,6222212,294,SF Residential,936.000000,9707.801990,200,0.0,0.0,57,2,8,59,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52,0.096417,33000.00,413268.2409,4558447.126,52.0,0.222860,16.038496,1378.496492,1119.0,1119.0,0.0,936.000000,1,0,0,0,0,0,0,0,0.0,1.91926,0.75272,0.775662,3.47225,4.87533,0.68058,0.46166,6407.0,2184.400000,2193.0,25.367793,0.0,0.0,93084.284723,93084.284723,58153.68243,63043.214090,51314.86389
1,103580,0.0,0,base,0,1943,1,80.235589,0.0,1.0,1.0,468869,762.000000,0.0,6222517,217,SF Residential,762.000000,7337.636717,200,0.0,0.0,57,2,7,42,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,0.103848,23000.00,417799.0966,4567252.757,72.0,0.168449,23.132538,1311.222624,19360.0,19360.0,0.0,762.000000,1,0,0,0,0,0,0,0,0.0,4.27822,1.89122,1.853717,1.41363,4.59179,1.23262,0.27137,10618.0,791.323213,821.0,26.166365,0.0,0.0,51945.000001,51945.000001,58153.68243,50800.000000,39993.99048
2,122314,0.0,0,base,0,2011,2,74.622907,0.0,1.0,1.0,544735,1254.356307,0.0,6225341,312,MF Residential,1254.356307,1218.829093,200,0.0,0.0,57,3,9,63,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1.029149,25000.00,420285.6256,4565731.264,4.0,0.027980,32.084091,1329.201011,3267.0,3267.0,0.0,1254.356307,0,1,0,0,0,0,0,0,0.0,4.38490,3.11208,3.050813,2.81490,3.26382,0.24758,0.13233,9717.0,405.160059,474.0,24.133877,0.0,0.0,0.000000,0.000000,42603.36000,33483.900000,42972.10000
3,154407,0.0,0,base,0,1994,1,55.859408,0.0,1.0,1.0,14751,5422.000000,0.0,6129850,1032,SF Residential,5422.000000,12623.790030,200,0.0,0.0,35,9,24,239,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,0.429507,180752.50,427751.3616,4515148.760,21.0,0.289802,0.673530,1871.942285,7002.0,7002.0,0.0,5422.000000,1,0,0,0,0,0,0,0,0.0,6.23179,2.86202,2.821646,4.38326,1.64086,0.06748,0.09761,7998.0,1573.270000,1583.0,23.035833,0.0,0.0,270300.000003,270300.000003,48208.29701,45755.066295,147237.66330
4,154684,0.0,0,base,0,1965,1,70.955552,0.0,1.0,1.0,14619,2626.000000,0.0,6141975,1032,SF Residential,2626.000000,13583.456040,200,1.0,0.0,35,9,24,239,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,0.193323,194493.38,426290.4076,4515484.012,50.0,0.311833,0.775872,1548.110380,1710.0,1125.0,0.0,2626.000000,1,0,0,0,0,0,0,0,0.0,5.31778,1.97403,1.935123,3.62918,2.40186,0.26217,0.32572,8512.0,1800.850000,1820.0,23.035833,0.0,0.0,306895.211228,306895.211228,48208.29701,45755.066295,147237.66330


In [5]:
newbuildings.head()

,building_id,unit_price_non_residential,non_residential_price,note,residential_price,year_built,building_type_id,res_price_per_sqft,non_residential_sqft,stories,residential_units,parcel_id,building_sqft,job_spaces,node_id,zone_id,general_type,sqft_per_unit,lot_size_per_unit,sqft_per_job,vacant_residential_units,vacant_job_spaces,county_id,distlrg_id,distmed_id,distsml_id,distmed_1,distmed_2,distmed_3,distmed_4,distmed_5,distmed_6,distmed_7,distmed_8,distmed_9,distmed_10,distmed_11,distmed_12,distmed_13,distmed_14,distmed_15,distmed_16,distmed_17,distmed_18,distmed_19,distmed_20,distmed_21,distmed_22,distmed_23,distmed_24,distmed_25,distmed_26,distmed_27,distmed_28,distmed_29,distmed_30,distmed_31,distmed_32,distmed_33,distmed_34,distmed_35,distmed_36,distmed_37,distmed_38,distmed_39,distmed_40,distmed_41,zone_1212,building_age,real_far,land_value,utmxi,utmyi,avg_building_age,parcel_acres,parcel_volume,elevation,volume_two_way,volume_two_way_nofwy,max_far,residential_sqft,is_sf,is_mf,is_industrial,is_retail,is_office,is_govt,is_mixeduse,is_other,airport,airport_distance,fwy_exit_dist,fwy_exit_dist_tdm_output,raildepot_dist,university_dist,trail_dist,stream_dist,rail_stn_dist,bus_rte_dist,bus_stop_dist,residential_sales_price_sqft,total_job_spaces,rail_depot,improvement_value,unit_price_residential,distlrg_med_inc,distmed_med_inc,distsml_med_inc
0,787134,0.0,0,simulated,0,2015,2,131.833811,0.0,4.0,77.0,42553,138453.330263,0.0,6148308,959,MF Residential,1798.095198,653.852799,200,77.0,0.0,35,8,23,229,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.75,882600.000,424000.7210,4512963.683,0.0,1.155800,128.989670,1293.979232,1717.0,1717.0,0.0,138453.330263,0,1,0,0,0,0,0,0,0.0,4.23031,0.76102,0.734617,3.50489,3.22927,1.11610,1.16495,383.0,225.767389,233.0,55.254502,0.0,0.0,0.0,0.0,36729.77964,36729.779640,39558.00000
1,787135,0.0,0,simulated,0,2015,2,71.170254,0.0,5.0,171.0,803110,552817.438000,0.0,29853719,1420,MF Residential,3232.850515,808.212629,200,171.0,0.0,35,11,30,336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.00,1413555.269,411600.0163,4497786.286,0.0,3.172736,25.574837,1534.046850,7626.0,7626.0,0.0,552817.438000,0,1,0,0,0,0,0,0,0.0,11.74713,5.70379,3.753002,2.87491,14.48068,4.06436,0.20244,40628.0,4300.270000,4300.0,16.457922,0.0,0.0,0.0,0.0,66332.88991,58855.727520,74854.72222
2,787136,0.0,0,simulated,0,2015,1,72.650680,0.0,1.0,1.0,117524,1949.960130,0.0,6117170,1213,SF Residential,1949.960130,4874.900324,200,1.0,0.0,35,9,25,284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.40,7500.000,425735.2343,4506288.398,0.0,0.111912,43.084824,1300.717446,436.0,436.0,0.0,1949.960130,1,0,0,0,0,0,0,0,0.0,7.64107,1.07682,1.050933,1.88489,4.66313,2.26454,0.04239,5052.0,586.847035,773.0,24.868794,0.0,0.0,0.0,0.0,48208.29701,39760.816670,35170.00000
3,787137,0.0,0,simulated,0,2015,2,123.638238,0.0,7.0,6.0,75311,9024.668801,0.0,28553079,1085,MF Residential,1504.111467,273.474812,200,6.0,0.0,35,9,24,253,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.50,2000.000,426651.7010,4512407.787,0.0,0.037669,144.500122,1321.634710,40.0,40.0,0.0,9024.668801,0,1,0,0,0,0,0,0,0.0,5.89219,2.02622,1.496897,3.86188,1.61753,1.30495,1.14645,949.0,443.690885,495.0,36.710576,0.0,0.0,0.0,0.0,48208.29701,45755.066295,32191.51515
4,787138,0.0,0,simulated,0,2015,2,52.760406,0.0,4.0,6.0,166164,20100.272355,0.0,6145728,900,MF Residential,3350.045393,2233.363595,200,6.0,0.0,35,7,22,213,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.50,103900.000,420930.7448,4501927.611,0.0,0.307626,69.794434,1303.310415,8714.0,8714.0,0.0,20100.272355,0,1,0,0,0,0,0,0,0.0,8.74081,0.93198,0.788508,3.05587,8.55982,0.74182,0.04958,11151.0,886.921548,889.0,21.420371,0.0,0.0,0.0,0.0,43465.62500,45524.500905,32000.00000


In [6]:
parcels.head()

,parcel_id,county_id,zone_id,parcel_acres,land_value,x,y,elevation,fwy_exit,airport,rail_depot,stream,trail,university,shape_area,volume_one_way,volume_two_way,airport_distance,fwy_exit_dist,raildepot_dist,university_dist,trail_dist,stream_dist,train_station,rail_stn_dist,bus_rte_dist,bus_stop,bus_stop_dist,volume_two_way_nofwy,distsml_id,distmed_id,distlrg_id,parent_parcel,CO_NAME,parcel_id_REMM,county_taz_id,utmxi,utmyi,city,zonal_ppa,cid_old,parcel_size,total_residential_units,total_job_spaces,total_sqft,zoned_du,zoned_du_underbuild,ave_sqft_per_unit,ave_unit_size,lot_size_per_unit,oldest_building,node_id,max_far,max_dua,max_height,residential_purchase_price_sqft,residential_sales_price_sqft,building_purchase_price_sqft,building_purchase_price,redev_friction,parcel_volume,gridID,agriculture,land_cost,distlrg_res_shift,distlrg_nonres_shift,avg_building_age,fwy_exit_dist_tdm_output
0,741871,49,2111,0.197968,35700.00,1531239.186,7330722.151,1447.537450,0.0,0.0,0.0,1.0,0.0,0.0,8.623484e+03,992.0,1942.0,18.02655,0.85831,12.61234,14.73993,0.09764,0.0,0.0,46115.0,2382.31,0.0,4575.0,1942.0,487,38,13,741871,UTAH,741871,492111,424331.7633,4477427.732,Lehi,176926.527483,4,8.623484e+03,1.0,0.0,2434.0,1,0.0,3299.525391,3299.525391,8.623484e+03,2005.0,44132976,NaN,6.0,999,62.075298,22.017656,62.075298,151091.274955,10.0,7.245125,33270.0,0.0,1.867913e+06,1.0,1.0,10.0,0.763505
1,579822,49,2873,37.717834,56614.32,1535437.548,7341595.932,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,1.642989e+06,3564.0,7135.0,19.48828,1.32984,10.63629,15.94851,0.23207,0.0,0.0,34220.0,4930.50,0.0,4930.0,7135.0,0,0,0,579822,UTAH,579822,492873,425629.9856,4480733.739,Utah County,140869.334754,4,1.642989e+06,0.0,0.0,0.0,0,0.0,3153.817058,3153.817058,1.642989e+06,9999.0,31696755,NaN,NaN,999,60.719574,4.938000,60.719574,0.000000,10.0,0.000000,35100.0,0.0,5.661432e+04,NaN,NaN,-99.0,1.282226
2,579853,49,2117,6.791235,81500.00,1534101.235,7334630.179,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,2.958262e+05,0.0,0.0,18.42054,1.66976,11.91035,15.00142,0.55081,0.0,0.0,41888.0,6770.95,0.0,8334.0,0.0,487,38,13,579853,UTAH,579853,492117,425210.6619,4478613.554,Lehi,199096.575130,4,2.958262e+05,0.0,0.0,0.0,33,33.0,3860.335791,3860.335791,2.958262e+05,9999.0,37189762,NaN,4.8,999,51.340155,17.843868,51.340155,0.000000,10.0,0.073485,33936.0,0.0,8.150000e+04,1.0,1.0,-99.0,1.614241
3,640185,49,2073,1.372797,112100.00,1538253.186,7322787.258,1445.761763,0.0,0.0,0.0,1.0,0.0,0.0,5.979902e+04,2495.0,5043.0,16.06753,0.60111,14.23267,12.73670,0.04966,0.0,0.0,54118.0,2627.34,0.0,2739.0,5043.0,482,38,13,640185,UTAH,640185,492073,426455.1775,4474997.689,Lehi,169098.553773,4,5.979902e+04,1.0,0.0,5499.0,3,2.0,2225.546469,2225.546469,5.979902e+04,1981.0,33881743,NaN,2.5,999,51.545684,18.188987,51.545684,283449.714089,10.0,13.470624,31949.0,0.0,3.955497e+06,1.0,1.0,34.0,0.547433
4,682698,49,2119,0.163751,43000.00,1548648.398,7336198.043,1766.927829,0.0,0.0,0.0,1.0,0.0,0.0,7.132996e+03,3632.0,7133.0,17.61362,3.74880,12.20122,13.84603,0.37882,0.0,0.0,43311.0,9997.32,0.0,11170.0,7133.0,488,39,14,682698,UTAH,682698,492119,429646.0053,4479065.962,Draper,254583.603528,4,7.132996e+03,1.0,0.0,3046.0,0,0.0,3584.635135,3584.635135,7.132996e+03,2006.0,58252499,NaN,0.2,999,55.862249,19.042765,55.862249,170156.411593,10.0,9.860687,34117.0,0.0,2.131564e+06,1.0,1.0,9.0,2.176557


In [7]:
# Show the size of the parcels table
parcels.shape

(841684, 68)

In [8]:
# # get existing buildings before simulation for that year
# existing_buildings = allbuildings[(allbuildings.year_built <= year) & (allbuildings.note == "base")]

In [9]:
# # get base buildings
# base_buildings = allbuildings[allbuildings.note == "base"]

# # get list of parcels that have a building on them in base
# ids_has_buildings = list(set(base_buildings["parcel_id"].to_list()))

# # filter to parcels that have building, probably should assign an attribute to thsi
# parcels_with_buildings_base = parcels[~parcels['parcel_id'].isin(ids_has_buildings)].copy()

### Establish baseline for current year

In [10]:
# get base buildings
base_buildings = allbuildings[allbuildings.note == "base"]

# get list of parcels that have a building on them in base
ids_has_buildings = list(set(base_buildings["parcel_id"].to_list()))

In [11]:
# subset parcel table to relevant fields
parcels_base = parcels[['parcel_id', 'parcel_id_REMM', 'county_id', 'zone_id','parcel_acres',
                        'land_value', 'land_cost', 'max_far', 'max_dua']].copy()

In [12]:
# in the base dataset, these parcels have buildings on them
parcels_with_buildings_base = parcels_base[parcels_base['parcel_id'].isin(ids_has_buildings)].copy()

# these parcels do NOT start with buildings on them
parcels_without_buildings_base = parcels_base[~parcels_base['parcel_id'].isin(ids_has_buildings)].copy()

In [13]:
# create column that indicates if parcel has a building
parcels_base['hasBuildings'] = 0

#
parcels_base.loc[parcels_base['parcel_id_REMM'].isin(ids_has_buildings), 'hasBuildings'] = 1

In [14]:
# get simulated buildings
simulated_buildings = allbuildings[allbuildings.note == "simulated"]


In [15]:
# quick test to see if parcel id and parcel id remm or the same
x = np.where(parcels['parcel_id']==parcels['parcel_id_REMM'],                                            'yes', 'no')
pd.DataFrame(x).value_counts()

yes    841684
dtype: int64

In [16]:
# summarize building attributes by parcel
base_buildings2 = base_buildings[['parcel_id','residential_units', 'job_spaces', 'building_sqft', 'residential_sqft']].copy()
buildings_by_parcel = base_buildings2.groupby('parcel_id').sum()

building_count = pd.DataFrame(base_buildings2['parcel_id'].value_counts()).reset_index()
building_count.columns = ['parcel_id', 'building_count']

buildings_by_parcel.merge(building_count, left_on='parcel_id', right_on='parcel_id', how='left')

,parcel_id,residential_units,job_spaces,building_sqft,residential_sqft,building_count
0,44,0.0,39.0,14586.0,0.0,1
1,152,1.0,0.0,2198.0,2198.0,1
2,215,0.0,260.0,97240.0,0.0,1
3,278,0.0,12.0,4488.0,0.0,1
4,308,0.0,14.0,13650.0,0.0,1
...,...,...,...,...,...,...
615470,871288,1.0,0.0,3269.0,3269.0,1
615471,872441,1.0,0.0,1680.0,1680.0,1
615472,875208,1.0,0.0,2971.0,2971.0,1
615473,882712,0.0,1.0,1050.0,0.0,1


In [17]:
b = pd.read_csv(r"E:\Projects\REMM2_For_Python3\REMMRun\run131year2050allbuildings.csv").reset_index()

In [18]:
b.head()

,index,building_id,non_residential_sqft,year_built,building_type_id,stories,residential_units,note,res_price_per_sqft,parcel_id,non_residential_price,residential_price,unit_price_non_residential,building_sqft,job_spaces,node_id,zone_id,general_type,sqft_per_unit,lot_size_per_unit,sqft_per_job,vacant_residential_units,vacant_job_spaces,county_id,distlrg_id,distmed_id,distsml_id,distmed_1,distmed_2,distmed_3,distmed_4,distmed_5,distmed_6,distmed_7,distmed_8,distmed_9,distmed_10,distmed_11,distmed_12,distmed_13,distmed_14,distmed_15,distmed_16,distmed_17,distmed_18,distmed_19,distmed_20,distmed_21,distmed_22,distmed_23,distmed_24,distmed_25,distmed_26,distmed_27,distmed_28,distmed_29,distmed_30,distmed_31,distmed_32,distmed_33,distmed_34,distmed_35,distmed_36,distmed_37,distmed_38,distmed_39,distmed_40,distmed_41,zone_1212,building_age,real_far,land_value,utmxi,utmyi,avg_building_age,parcel_acres,parcel_volume,elevation,volume_two_way,volume_two_way_nofwy,max_far,residential_sqft,is_sf,is_mf,is_industrial,is_retail,is_office,is_govt,is_mixeduse,is_other,airport,airport_distance,fwy_exit_dist,fwy_exit_dist_tdm_output,raildepot_dist,university_dist,trail_dist,stream_dist,rail_stn_dist,bus_rte_dist,bus_stop_dist,residential_sales_price_sqft,total_job_spaces,rail_depot,improvement_value,unit_price_residential,distlrg_med_inc,distmed_med_inc,distsml_med_inc,households,population,jobs,jobs1,jobs2,jobs3,jobs4,jobs5,jobs6,jobs7,jobs8,jobs9,jobs10
0,0,65788,0.0,1963,1,1.0,1.0,base,83.029856,487357,0,0,0.0,936.000000,0.0,6222212,294,SF Residential,936.000000,9707.801990,200,0.0,0.0,57,2,8,59,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87,0.096417,33000.00,413268.2409,4558447.126,87.0,0.222860,15.185373,1378.496492,1119.0,1119.0,0.0,936.000000,1,0,0,0,0,0,0,0,0.0,1.91926,0.75272,0.775662,3.47225,4.87533,0.68058,0.46166,6407.0,2184.400000,2193.0,24.612637,0.0,0.0,77715.944874,77715.944874,58216.97627,61085.35198,50463.69444,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,103580,0.0,1943,1,1.0,1.0,base,81.079709,468869,0,0,0.0,762.000000,0.0,6222517,217,SF Residential,762.000000,7337.636717,200,0.0,0.0,57,2,7,42,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,107,0.103848,23000.00,417799.0966,4567252.757,107.0,0.168449,27.931253,1311.222624,19360.0,19360.0,0.0,762.000000,1,0,0,0,0,0,0,0,0.0,4.27822,1.89122,1.853717,1.41363,4.59179,1.23262,0.27137,10618.0,791.323213,821.0,23.952715,0.0,0.0,61782.738495,61782.738495,58216.97627,52146.21337,42037.31959,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,122314,0.0,2011,2,1.0,1.0,base,59.848710,544735,0,0,0.0,1254.356307,0.0,6225341,312,MF Residential,1254.356307,1218.829093,200,0.0,0.0,57,3,9,63,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,1.029149,25000.00,420285.6256,4565731.264,39.0,0.027980,43.801609,1329.201011,3267.0,3267.0,0.0,1254.356307,0,1,0,0,0,0,0,0,0.0,4.38490,3.11208,3.050813,2.81490,3.26382,0.24758,0.13233,9717.0,405.160059,474.0,20.719832,0.0,0.0,75480.161524,75480.161524,45000.00000,40000.00000,44718.00000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,154407,0.0,1994,1,1.0,1.0,base,54.340386,14751,0,0,0.0,5422.000000,0.0,6129850,1032,SF Residential,5422.000000,12623.790030,200,0.0,0.0,35,9,24,239,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56,0.429507,180752.50,427751.3616,4515148.760,56.0,0.289802,0.766268,1871.942285,7002.0,7002.0,0.0,5422.000000,1,0,0,0,0,0,0,0,0.0,6.23179,2.86202,2.821646,4.38326,1.64086,0.06748,0.09761,7998.0,1573.270000,1583.0,21.694050,0.0,0.0,294654.510813,294654.510813,47187.55833,44722.78996,128786.38040,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,154684,0.0,1965,1,1.0,1.0,base,69.256787,14619,0,0,0.0,2626.000000,0.0,6141975,1032,SF Residential,2626.000000,13583.456040,200,1.0,0.0,35,9,24,239,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

### Check if buildings  in a parcel overwrite each other, if a new one is developed

In [ ]:
x = pd.DataFrame(b.groupby('parcel_id').agg({'year_built': ['count', 'min', 'max']}))

In [20]:
x.to_csv(r'.\Outputs\buildings_groupby.csv')

### Evaluate if parcel is developable

In [ ]:
zoning

In [ ]:
# indicate which parcels are developable
zoning.loc[(zoning['type1'] == 'f') & (zoning['type2'] == 'f') & 
           (zoning['type3'] == 'f') & (zoning['type4'] == 'f') &
           (zoning['type5'] == 'f') & (zoning['type6'] == 'f'), 'developable'] = 0

zoning['developable'].fillna(1, inplace=True)
zoning

### Aggregate building acreage and value by parcel id and type

#### Types
- SF Residential
- MF Residential
- Govt
- Industrial
- Mixed Use
- Retail
- Other

In [55]:
allbuildings.head()

,building_id,unit_price_non_residential,non_residential_price,note,residential_price,year_built,building_type_id,res_price_per_sqft,non_residential_sqft,stories,residential_units,parcel_id,building_sqft,job_spaces,node_id,zone_id,general_type,sqft_per_unit,lot_size_per_unit,sqft_per_job,vacant_residential_units,vacant_job_spaces,county_id,distlrg_id,distmed_id,distsml_id,distmed_1,distmed_2,distmed_3,distmed_4,distmed_5,distmed_6,distmed_7,distmed_8,distmed_9,distmed_10,distmed_11,distmed_12,distmed_13,distmed_14,distmed_15,distmed_16,distmed_17,distmed_18,distmed_19,distmed_20,distmed_21,distmed_22,distmed_23,distmed_24,distmed_25,distmed_26,distmed_27,distmed_28,distmed_29,distmed_30,distmed_31,distmed_32,distmed_33,distmed_34,distmed_35,distmed_36,distmed_37,distmed_38,distmed_39,distmed_40,distmed_41,zone_1212,building_age,real_far,land_value,utmxi,utmyi,avg_building_age,parcel_acres,parcel_volume,elevation,volume_two_way,volume_two_way_nofwy,max_far,residential_sqft,is_sf,is_mf,is_industrial,is_retail,is_office,is_govt,is_mixeduse,is_other,airport,airport_distance,fwy_exit_dist,fwy_exit_dist_tdm_output,raildepot_dist,university_dist,trail_dist,stream_dist,rail_stn_dist,bus_rte_dist,bus_stop_dist,residential_sales_price_sqft,total_job_spaces,rail_depot,improvement_value,unit_price_residential,distlrg_med_inc,distmed_med_inc,distsml_med_inc
0,65788,0.0,0,base,0,1963,1,84.432841,0.0,1.0,1.0,487357,936.000000,0.0,6222212,294,SF Residential,936.000000,9707.801990,200,0.0,0.0,57,2,8,59,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52,0.096417,33000.00,413268.2409,4558447.126,52.0,0.222860,16.038496,1378.496492,1119.0,1119.0,0.0,936.000000,1,0,0,0,0,0,0,0,0.0,1.91926,0.75272,0.775662,3.47225,4.87533,0.68058,0.46166,6407.0,2184.400000,2193.0,25.367793,0.0,0.0,93084.284723,93084.284723,58153.68243,63043.214090,51314.86389
1,103580,0.0,0,base,0,1943,1,80.235589,0.0,1.0,1.0,468869,762.000000,0.0,6222517,217,SF Residential,762.000000,7337.636717,200,0.0,0.0,57,2,7,42,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,0.103848,23000.00,417799.0966,4567252.757,72.0,0.168449,23.132538,1311.222624,19360.0,19360.0,0.0,762.000000,1,0,0,0,0,0,0,0,0.0,4.27822,1.89122,1.853717,1.41363,4.59179,1.23262,0.27137,10618.0,791.323213,821.0,26.166365,0.0,0.0,51945.000001,51945.000001,58153.68243,50800.000000,39993.99048
2,122314,0.0,0,base,0,2011,2,74.622907,0.0,1.0,1.0,544735,1254.356307,0.0,6225341,312,MF Residential,1254.356307,1218.829093,200,0.0,0.0,57,3,9,63,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1.029149,25000.00,420285.6256,4565731.264,4.0,0.027980,32.084091,1329.201011,3267.0,3267.0,0.0,1254.356307,0,1,0,0,0,0,0,0,0.0,4.38490,3.11208,3.050813,2.81490,3.26382,0.24758,0.13233,9717.0,405.160059,474.0,24.133877,0.0,0.0,0.000000,0.000000,42603.36000,33483.900000,42972.10000
3,154407,0.0,0,base,0,1994,1,55.859408,0.0,1.0,1.0,14751,5422.000000,0.0,6129850,1032,SF Residential,5422.000000,12623.790030,200,0.0,0.0,35,9,24,239,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,0.429507,180752.50,427751.3616,4515148.760,21.0,0.289802,0.673530,1871.942285,7002.0,7002.0,0.0,5422.000000,1,0,0,0,0,0,0,0,0.0,6.23179,2.86202,2.821646,4.38326,1.64086,0.06748,0.09761,7998.0,1573.270000,1583.0,23.035833,0.0,0.0,270300.000003,270300.000003,48208.29701,45755.066295,147237.66330
4,154684,0.0,0,base,0,1965,1,70.955552,0.0,1.0,1.0,14619,2626.000000,0.0,6141975,1032,SF Residential,2626.000000,13583.456040,200,1.0,0.0,35,9,24,239,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,0.193323,194493.38,426290.4076,4515484.012,50.0,0.311833,0.775872,1548.110380,1710.0,1125.0,0.0,2626.000000,1,0,0,0,0,0,0,0,0.0,5.31778,1.97403,1.935123,3.62918,2.40186,0.26217,0.32572,8512.0,1800.850000,1820.0,23.035833,0.0,0.0,306895.211228,306895.211228,48208.29701,45755.066295,147237.66330


In [54]:
x = pd.DataFrame(allbuildings.groupby(['parcel_id', 'general_type'])[['building_sqft']].sum()).reset_index()
x

,parcel_id,general_type,building_sqft
0,44,Retail,14586.000000
1,152,SF Residential,2198.000000
2,215,Retail,97240.000000
3,278,Retail,4488.000000
4,308,Industrial,13650.000000
...,...,...,...
619346,885552,SF Residential,53745.718750
619347,885553,SF Residential,108900.000000
619348,885563,SF Residential,26935.363880
619349,885905,SF Residential,39306.578440


In [66]:
# check if some parcels have multiple building types
x['parcel_id'].value_counts().head(10)

600219    3
478456    3
550841    3
491355    3
445737    2
461542    2
557103    2
495428    2
554255    2
508641    2
Name: parcel_id, dtype: int64

In [72]:
x_pivot = x.pivot(index='parcel_id', columns='general_type', values='building_sqft').reset_index().fillna(0)
x_pivot.head(10)

general_type,parcel_id,Govt,Industrial,MF Residential,Mixed Use,Office,Other,Retail,SF Residential
0,44,0.0,0.0,0.0,0.0,0.0,0.0,14586.0,0.0
1,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2198.0
2,215,0.0,0.0,0.0,0.0,0.0,0.0,97240.0,0.0
3,278,0.0,0.0,0.0,0.0,0.0,0.0,4488.0,0.0
4,308,0.0,13650.0,0.0,0.0,0.0,0.0,0.0,0.0
5,563,0.0,0.0,1330.0,0.0,0.0,0.0,0.0,0.0
6,1007,0.0,280800.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1051,0.0,4875.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1075,0.0,0.0,0.0,0.0,129765.0,0.0,0.0,0.0
9,1080,0.0,262275.0,0.0,0.0,0.0,0.0,0.0,0.0


general_type,parcel_id,Govt,Industrial,MF Residential,Mixed Use,Office,Other,Retail,SF Residential
0,44,0.0,0.0,0.0,0.0,0.0,0.0,14586.0,0.000000
1,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2198.000000
2,215,0.0,0.0,0.0,0.0,0.0,0.0,97240.0,0.000000
3,278,0.0,0.0,0.0,0.0,0.0,0.0,4488.0,0.000000
4,308,0.0,13650.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
617199,885552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53745.718750
617200,885553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108900.000000
617201,885563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26935.363880
617202,885905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39306.578440


In [2]:
jobs = pd.read_csv(r"E:\REMM\Base_Year_Data_2019\jobs.csv")
jobs.groupby(['building_id', 'sector_id']).count().reset_index()

,building_id,sector_id,jobs_id,cid
0,1352,1,25,25
1,1352,6,23,23
2,1352,7,102,102
3,1352,9,40,40
4,1570,7,1,1
...,...,...,...,...
69354,787130,7,1,1
69355,787130,9,2,2
69356,787131,7,1,1
69357,787132,6,2,2


In [5]:
pd.pivot_table(jobs, values='cid', index = 'building_id', columns='sector_id', aggfunc='count').reset_index()

sector_id,building_id,1,3,4,5,6,7,9,10
0,1352,25.0,NaN,NaN,NaN,23.0,102.0,40.0,NaN
1,1570,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,1614,NaN,NaN,NaN,NaN,1.0,7.0,1.0,8.0
3,1716,34.0,NaN,NaN,NaN,73.0,24.0,1.0,NaN
4,1850,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
31876,787128,7.0,1.0,5.0,8.0,1.0,6.0,1.0,7.0
31877,787129,NaN,36.0,NaN,NaN,NaN,1.0,NaN,NaN
31878,787130,NaN,4.0,NaN,NaN,NaN,1.0,2.0,NaN
31879,787131,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
